### Map optimization output back to shapefile
The functions below prepare the data for the process of mapping the results back to their original shapefiles by implementing a "table join" between the outputs .txt and the attribute table of the shapefile. This will allow the visualization of accessibility **after** optimization.
The process for mapping the optimization results is as follows:
    1. we need to filter the snapped destinations file by the IDs from the optimal selected facilities that were generated from the optimization analysis
    2. we need to slice the OD matrix by the new destination file, so that only the columns that represent the optimal selected facilities remain
    3. we map the OD matrix back to the origins shapefile as in Step 4

In [193]:
import pandas as pd
import os, sys
sys.path.append(r'../../../GOSTNets/GOSTNets')
import GOSTnet as gn
import importlib
importlib.reload(gn)
import geopandas as gpd
from shapely.wkt import loads
import numpy as np

networkx version: 2.2 
osmnx version: 0.9 


In [194]:
# define the paths to the origins, destinations, optimization result
pth = r'../../../../lima_optimization_output'
WGS = {'init':'epsg:4326'}
measure_crs = {'init':'epsg:32718'}

In [195]:
# read in the original destination file (from Step3)
destinations_df = pd.read_csv(os.path.join(pth, 'destinations_snapped.csv'))

In [196]:
destinations_df[:3]

,Unnamed: 0,Field1,departamen,provincia,distrito,categoria,Lat,Lon,O_ID,geometry,NN,NN_dist
0,0,337,LIMA,LIMA,VILLA EL SALVADOR,12,-12.248749,-76.930702,337,POINT (-76.93070221000001 -12.24874878),6691,17.668383
1,1,338,LIMA,LIMA,VILLA EL SALVADOR,12,-12.208811,-76.955727,338,POINT (-76.95572661999999 -12.20881081),6048,16.737332
2,2,339,LIMA,LIMA,VILLA EL SALVADOR,14,-12.230375,-76.923637,339,POINT (-76.92363739 -12.23037529),3914,46.775556


In [197]:
# include the path of the optimization results file *.txt and open the file
Rfile = os.path.join(pth, 'results.txt')
with open (Rfile, 'r') as re:
    results = re.readlines()
results =[(el.strip()) for el in results]
results = [int(i) for i in results]
results[:5]

[2048, 3409, 4154, 6107]

In [198]:
# filter optimal destinations by the result of the optimization process
destinations = destinations_df[destination_df['NN'].isin(results)]
destinations[:100]

,Unnamed: 0,Field1,departamen,provincia,distrito,categoria,Lat,Lon,O_ID,geometry,NN,NN_dist
5,5,342,LIMA,LIMA,VILLA EL SALVADOR,12,-12.224347,-76.922035,342,POINT (-76.92203522 -12.22434711),4154,56.753151
7,7,344,LIMA,LIMA,VILLA EL SALVADOR,12,-12.201385,-76.948044,344,POINT (-76.94804382 -12.2013855),3409,23.288640
13,13,351,LIMA,LIMA,VILLA EL SALVADOR,12,-12.228789,-76.942665,351,POINT (-76.9426651 -12.22878933),2048,19.978944
15,15,353,LIMA,LIMA,VILLA EL SALVADOR,12,-12.213234,-76.944588,353,POINT (-76.94458770999999 -12.21323395),6107,19.211260


In [199]:
# write optimal destinations file back to .csv for mapping purposes
destinations.to_csv(os.path.join(pth, 'optimal_destinations.csv'))

In [200]:
# read OD matrix
OD = pd.read_csv(os.path.join(pth, r'saved_OD.csv'), sep=',')
OD = OD.rename(columns = {'Unnamed: 0':'NN'})
OD = OD.set_index('NN')

In [202]:
# transform the destination nodes into "string" type to be used as column headers
results_s = [str(x) for x in results]
# filter OD matrix by results
OD = OD[results_s]
OD[:3]

,2048,3409,4154,6107
NN,,,,
6147,1020.721567,1879.396717,819.749517,1584.512721
2052,558.842693,1019.720100,1591.881656,544.361984
3,448.775456,1815.601711,1380.165021,1236.519452


In [203]:
# adding a column that has the minimum time to reach each origin's nearest facility
OD['Minimum'] = OD.loc[:,:].min(axis=1)
OD[:3]

,2048,3409,4154,6107,Minimum
NN,,,,,
6147,1020.721567,1879.396717,819.749517,1584.512721,819.749517
2052,558.842693,1019.720100,1591.881656,544.361984,544.361984
3,448.775456,1815.601711,1380.165021,1236.519452,448.775456


In [204]:
### PREPARE ORIGIN FILE 
# set index to nearest node to match on the travel time from that node to nearest destination (OD['Minimum'])
origins_df = pd.read_csv(os.path.join(pth, 'origins_snapped.csv'))
# orig = orig.rename({'ID':'OBJECTID'}, axis = 1) # rename ID column to 'OBJECTID'
origins_df = origins_df.set_index('NN') # used for matching on the OD matrix
origins_df[:3]

,Unnamed: 0,O_ID,NC_CLASS,Shape_Leng,Shape_Area,ORIG_FID,Population,Lat,Lon,geometry,NN_dist
NN,,,,,,,,,,,
6528,0,1,3.0,0.003205,2.860800e-07,0,1078.0,-12.24839,-76.91749,POINT (-76.91749 -12.24839),21.347245
5270,1,2,3.0,0.008010,2.565700e-06,1,2374.0,-12.24319,-76.92767,POINT (-76.92766999999999 -12.24319),37.506920
1921,2,3,3.0,0.010808,2.141710e-06,2,367.0,-12.24444,-76.91685,POINT (-76.91685 -12.24444),6.260416


In [205]:
# join works on the index as default 
#join = origins_df.join(OD)

# make sure the index of both dataframes are the same before you try to copy the Minimum column to the origins_df
origins_df['Minimum'] = OD['Minimum']
#origins_df

In [206]:
# add on walktime (kph) for time computation
walkspeed = 4
origins_df['walktime'] = origins_df['NN_dist'] / 1000 / walkspeed * 3600
origins_df['total_time_min'] = (origins_df['walktime'] + origins_df['Minimum']) / 60
###   OR   ###
#compute total distance: NN_dist + dist
#origins_df['total_distance'] = (origins_df['NN_dist'] + origins_df['Minimum'])

In [207]:
# read in the origins in shapefile format
fp = r'../../Peru_Lima_Optimization/test_data'
shpfil = gpd.read_file(os.path.join(fp, 'VillaElSalvador_urban.shp'))

In [208]:
# make sure the index of both dataframes are the same before you try to copy a column over
shpfil = shpfil.set_index('OBJECTID')
origins_df = origins_df.set_index('O_ID')

In [209]:
shpfil['total_time_min'] = origins_df['total_time_min']

In [210]:
# Prep output shapefile, match on visualization column, save
#shpfil['total_time'] = join['total_time']
shpfil = shpfil.to_crs({'init':'epsg:4326'})
shpfil.to_file(os.path.join(pth, 'VES_origins_w_time.shp'), driver = 'ESRI Shapefile')